# Structuring programs using types and dispatch

Previously we specified an algorithm directly using a function. In many Julia packages it is common to use *dispatch* to do this. 

In Julia, **dispatch** refers to choosing which **method** (version) of a function to use, according to the type of the arguments. (**Multiple dispatch** is when the types of several different arguments are involved.)

Let's define some types to represent different differentiation methods.

#### Exercise 1

1. Define an abstract type `DifferentiationAlgorithm`.


2. Define subtypes `FiniteDifference`, `MyAutoDiff` (for our implementation) and `AutoDiff` (for the `ForwardDiff` implementation).


3. Implement the function `derivative(f, x, method)` using **dispatch**: for each of the three types, define a version of this function in which `method` is specified to be of that type by using the type annotation operator `::`.


4. Verify that these work by writing tests for them.

#### Exercise 2

1. Write a version of the Newton algorithm that takes an optional keyword argument `algorithm` specifying the differentiation algorithm to use.

## Functions as types

How could we differentiate the `sin` function? Of course, we can do it using e.g. finite differences, but in cases like this we actually know the exact, analytical derivative, in this case `cos`, i.e $\sin'(x) = \cos(x)$.
Is there are a way to tell Julia to use this directly? I.e. if `f` is equal to the `sin` function, then we should make a special version of our `derivative` function.

It turns out that there is, using dispatch, by checking what *type* the `sin` function has:

#### Exercise 3

1. Use `typeof` to find the type of the `sin` function.


2. Use this to make a special dispatch for `derivative(sin, x)`.

The package [`ChainRules.jl`](https://github.com/JuliaDiff/ChainRules.jl) contains definitions like this and is used inside `ForwardDiff.jl` and other packages that need to know the derivatives of functions.

## Representing a problem using a type

A root-finding problem requires several pieces of information: a function, a starting point, a root-finding algorithm to use, possibly a derivative, etc. We could just pass all of these as arguments to a function.
An alternative is to wrap up the various pieces of information into a new composite type.

#### Exercise 4

1. Make a `RootAlgorithm` abstract type.


2. Make a `Newton` subtype.


3. Make `Newton` a *callable* type using the syntax

    ```
    function (algorithm::Newton)(x)
        ...
     end
     
     ```
    
    This means that you will be able to call an object of type `Newton` as if it were a standard function, by passing in an argument. (You can add further arguments as necessary.)


4. Make a `RootProblem` type that contains the necessary information for a root problem. Do not specify types for the fields in this type yet. One of the fields should be called `algorithm`. 


5. Make a function `solve` that calls the field `algorithm` as a function.

### Type-based dispatch

So far we are not using the types to their full advantage: we wish to *dispatch* on the *type* of `algorithm`. We can do so by **parametrising the type**:

In [1]:
struct RootProblem2{T<:RootAlgorithm}
    ...
    algorithm::T
end

When we create an object of type `RootProblem2`, we will get a specialised version with the correct type. We can now use that in dispatch:

In [ ]:
solve(prob::RootProblem2{Newton}) = ...

#### Exercise 5

1. Implement this.


2. Put everything together to be able to solve a root problem using a particular derivative algorithm.

## Other uses of types

Other examples of different usages of types include:

- [`ModelingToolkit.jl`](https://github.com/JuliaDiffEq/ModelingToolkit.jl)

    Types are introduced to represent variables and operations. In this way it is relatively simple to build up a way to output symbolic expressions from standard Julia functions.
    
    
- https://github.com/MikeInnes/diff-zoo defines types to represent "tapes" recording sequences of operations. This is a precursor to tools such as [`Zygote.jl`](https://github.com/FluxML/Zygote.jl), which performs advanced automatic differentiation on code at a lower level.

### Traits

An important use of types that we have not addressed here is to define **traits**. These are labels that can be assigned to different types that may then be dispatched on, even if those types are not in a Julia type hierarchy.

See e.g. the implementation in [`SimpleTraits.jl`](https://github.com/mauro3/SimpleTraits.jl).